In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')

In [4]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
  df,
  cutoff_date=datetime(2023, 6, 1, 0, 0, 0),
  target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape}')
print(f'{y_train.shape}')
print(f'{X_test.shape}')
print(f'{y_test.shape}')

(32595, 674)
(32595,)
(56710, 674)
(56710,)


In [5]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = KFold(n_splits=3)
    scores = []
        
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [6]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2024-04-22 18:37:43,543] A new study created in memory with name: no-name-cfdbeee6-78b9-41b6-903d-79a1355a9630
[I 2024-04-22 18:37:49,516] Trial 0 finished with value: 2.6720555405244184 and parameters: {'num_leaves': 177, 'feature_fraction': 0.227542002019144, 'bagging_fraction': 0.48556607135340446, 'min_child_samples': 94}. Best is trial 0 with value: 2.6720555405244184.
[I 2024-04-22 18:38:03,706] Trial 1 finished with value: 2.668259487193288 and parameters: {'num_leaves': 185, 'feature_fraction': 0.7414965584498614, 'bagging_fraction': 0.43053722643226283, 'min_child_samples': 86}. Best is trial 1 with value: 2.668259487193288.
[I 2024-04-22 18:38:12,917] Trial 2 finished with value: 2.5888041456898043 and parameters: {'num_leaves': 144, 'feature_fraction': 0.5092338572602877, 'bagging_fraction': 0.9512789648012225, 'min_child_samples': 32}. Best is trial 2 with value: 2.5888041456898043.
[I 2024-04-22 18:38:17,760] Trial 3 finished with value: 2.6108043794899563 and parameter

In [7]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 144, 'feature_fraction': 0.5092338572602877, 'bagging_fraction': 0.9512789648012225, 'min_child_samples': 32}


In [8]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.5092338572602877, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5092338572602877
[LightGBM] [Warning] bagging_fraction is set=0.9512789648012225, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9512789648012225
[LightGBM] [Warning] feature_fraction is set=0.5092338572602877, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5092338572602877
[LightGBM] [Warning] bagging_fraction is set=0.9512789648012225, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9512789648012225
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.056084 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 156147
[LightGBM] [Info] Number of data points in the train set: 32595, number of used features: 675
[LightGBM] [Info] Start training from score 11.288664


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x00000232BBB42C10>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.9512789648012225,
                               feature_fraction=0.5092338572602877,
                               min_child_samples=32, num_leaves=144))])

In [9]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.5092338572602877, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5092338572602877
[LightGBM] [Warning] bagging_fraction is set=0.9512789648012225, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9512789648012225
test_mae=2.9953


In [12]:
from src.plot import plot_one_sample

plot_one_sample(
  features=X_test, 
  targets=y_test,
  example_id=50000,
  predictions=pd.Series(predictions)
)